# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys_google import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
path = "../WeatherPy/WeatherPy.csv"
df = pd.read_csv(path)
df.drop('Unnamed: 0', axis=1, inplace=True)
df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Lagoa,39.0500,-27.9833,291.06,76,100,8.35,PT,1666491111
1,Bulawayo,-20.1500,28.5833,293.32,50,88,5.04,ZW,1666491111
2,Tuatapere,-46.1333,167.6833,291.37,47,100,2.92,NZ,1666491112
3,Ushuaia,-54.8000,-68.3000,280.96,66,100,1.03,AR,1666491091
4,Saint Paul Harbor,57.7900,-152.4072,276.88,87,100,4.12,US,1666491112
...,...,...,...,...,...,...,...,...,...
573,Talara,-4.5772,-81.2719,290.47,90,100,6.71,PE,1666491331
574,Lixouri,38.2019,20.4314,292.13,76,5,2.42,GR,1666491331
575,Caconda,-13.7333,15.0667,289.94,77,13,0.85,AO,1666491079
576,Orocué,4.7912,-71.3354,299.88,78,70,2.09,CO,1666491332


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [5]:
locations = df[["Latitude", "Longitude"]].astype(float)
humidity_percent = df["Humidity"].astype(float)

In [6]:
heat_layer = gmaps.heatmap_layer(locations, weights=humidity_percent, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
fit_df = df.loc[(df['Humidity']>40)& (df['Humidity']<50)& (df['Cloudiness']<30)&(df["Max Temp"]<280)].dropna()
fit_df


,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
67,Genhe,50.7833,121.5167,273.28,48,7,1.83,CN,1666491136
131,Fareydūnshahr,32.9410,50.1210,279.57,49,0,0.54,IR,1666491158
193,Langxiang,46.9500,128.8833,277.12,48,21,1.73,CN,1666491183
477,Petropavlovsk-Kamchatskiy,53.0452,158.6483,277.48,42,20,2.00,RU,1666491039
558,Kalanguy,51.0167,116.5167,273.65,49,0,1.57,RU,1666491138


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
hotel_df = fit_df[["City","Country", "Latitude", "Longitude"]]
hotel_df['Hotel Name'] = ''
hotel_df

/Users/joshuasamuel/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Country,Latitude,Longitude,Hotel Name
67,Genhe,CN,50.7833,121.5167,
131,Fareydūnshahr,IR,32.9410,50.1210,
193,Langxiang,CN,46.9500,128.8833,
477,Petropavlovsk-Kamchatskiy,RU,53.0452,158.6483,
558,Kalanguy,RU,51.0167,116.5167,


In [15]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [23]:
for index, row in hotel_df.iterrows():
    lati = row['Latitude']
    long = row['Longitude']
    citi = row['City']
    params['location'] = f"{lati},{long}"
    response = requests.get(base_url, params=params).json()
    results = response['results']
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    except(KeyError, IndexError):
        print(f"No Hotel found for {citi} ")
                                              
            
hotel_df
    

/Users/joshuasamuel/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


No Hotel found for Kalanguy 


,City,Country,Latitude,Longitude,Hotel Name
67,Genhe,CN,50.7833,121.5167,The Root River And the Mountain Hotel
131,Fareydūnshahr,IR,32.9410,50.1210,هتل سوپل
193,Langxiang,CN,46.9500,128.8833,朗乡宾馆
477,Petropavlovsk-Kamchatskiy,RU,53.0452,158.6483,Nachalnik Kamchatki
558,Kalanguy,RU,51.0167,116.5167,


In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [26]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content =hotel_info )

# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…